# Energy Loss WML Deployment Recipe

This notebook demonstrates the use of AI Model Factory recipes to deploy energy loss pipeline to WML

## Import libraries

In [1]:
import json
import requests
import yaml

## Set Variables

model_uri - URI to fetch the model from \
deployment_name - name to set for the deployment

In [2]:
deployment_name = "energyloss_111523_v2_three_kpis_loss_rate_joe"

In [3]:
model_factory_config_file_name = "../config/model_info_three_kpis_loss_rate.yml"


with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

{'model_uri': 'runs:/794a004e0fae4a28a80fe456ed08ae3d/energyloss', 's3_uri': 's3://testdataupload/2/d443512c9ba54cc5993beb2a56ba5480/artifacts/energyloss', 'train_job_id': 'e7f53652-ae73-413f-8d3d-67af26eea92b'}


In [4]:
model_uri = model_factory_config['s3_uri']

In [5]:
print(model_uri)

s3://testdataupload/2/d443512c9ba54cc5993beb2a56ba5480/artifacts/energyloss


In [6]:
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"
credentials_path = "../../config/credentials/wml-credentials.json"

In [7]:
url = endpoint_url + "/deployment/wml/create"

## Create a request to call WML deployment recipe

In [8]:
payload = {
    'endpoint': 'electrical_transformer_deployments',
    'deployment_name': deployment_name,
    'model_uri': model_uri,
    'flavor': 'sklearn',
    'software_spec_name': 'modelfactory',
    
}

In [9]:
files={
      "wml_credentials_file": ('wml-credentials.json',open(credentials_path,'rb'),'application/json')
}

In [10]:
headers = {
  'accept': 'application/json'
}

In [11]:
response = requests.request("POST", url, headers=headers, data=payload, files=files)

In [12]:
print(response.text)

{"job_id":"3e527848-209b-4f39-b812-7a4d8f04db99","message":"WML deployment energyloss_111523_v2_three_kpis_loss_rate_joe submitted for creation"}


## Check the status of the job

In [13]:
job_id = json.loads(response.text)["job_id"]

In [14]:
url = endpoint_url + "/log" + f"/{job_id}"

In [15]:
response = requests.request("GET", url, headers=headers, data={}, files=[])

In [16]:
print(json.loads(response.text)["logs"])

recipe_file_path /tmp/ray/session_2023-11-16_01-58-51_212131_1/runtime_resources/working_dir_files/_ray_pkg_b55ff192fa810b59/Recipe.create
entrypoint_path /tmp/ray/session_2023-11-16_01-58-51_212131_1/runtime_resources/working_dir_files/_ray_pkg_b55ff192fa810b59/main.py
2023-11-16 02:10:49,002	INFO worker.py:1334 -- Using address 172.63.0.3:6379 set in the environment variable RAY_ADDRESS
2023-11-16 02:10:49,002	INFO worker.py:1452 -- Connecting to existing Ray cluster at address: 172.63.0.3:6379...
2023-11-16 02:10:49,009	INFO worker.py:1627 -- Connected to Ray cluster. View the dashboard at 172.63.0.3:8265 
(run_recipe_on_ray pid=144, ip=172.63.0.5) mlflow.experiment.name: 3e527848-209b-4f39-b812-7a4d8f04db99
(run_recipe_on_ray pid=144, ip=172.63.0.5) mlflow.experiment.id: 3
(run_recipe_on_ray pid=144, ip=172.63.0.5) 2023/11/16 02:10:49 INFO mlflow.tracking.fluent: Experiment with name '3e527848-209b-4f39-b812-7a4d8f04db99' does not exist. Creating a new experiment.
(run_recipe_on_ra

## Get the summary of the job

In [17]:
url = endpoint_url + "/summary" + f"/{job_id}"

In [18]:
response = requests.request("GET", url, headers=headers, data={}, files=[])

In [19]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"
    
    # Close the HTML code
    html_code += "</div>"
    
    return html_code

In [28]:
print_job_details(job_id, endpoint_url)

In [29]:
# Check the summary
summary_url = endpoint_url + "/summary/"
get_response = requests.get(summary_url+job_id, headers={})
summary_data = get_response.json()
print(summary_data)

{'job_id': '3e527848-209b-4f39-b812-7a4d8f04db99', 'status': 'DONE', 'summary': {'deployment_details': {'entity': {'asset': {'id': '9f3672e8-015b-439b-9ab6-24023315abe1', 'rev': '1'}, 'custom': {'AWS_ACCESS_KEY_ID': 'e6d28d7de535464da066c608726b4ee3', 'AWS_SECRET_ACCESS_KEY': '2fc6029cc5d799b6c572c0e730d6bc912c047cb1dddce409', 'MLFLOW_S3_ENDPOINT_URL': 'https://s3.us-south.cloud-object-storage.appdomain.cloud', 'MLFLOW_TRACKING_URI': 'http://mlflow:5000'}, 'deployed_asset_type': 'model', 'hardware_spec': {'id': 'b128f957-581d-46d0-95b6-8af5cd5be580', 'name': 'XXS', 'num_nodes': 1}, 'name': 'energyloss_111523_v2_three_kpis_loss_rate_joe', 'online': {}, 'space_id': 'a0521ce5-5547-41e2-8495-66012d7e8780', 'status': {'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'}, 'serving_urls': ['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811

In [52]:
summary_data

{'job_id': '3e527848-209b-4f39-b812-7a4d8f04db99',
 'status': 'DONE',
 'summary': {'deployment_details': {'entity': {'asset': {'id': '9f3672e8-015b-439b-9ab6-24023315abe1',
     'rev': '1'},
    'custom': {'AWS_ACCESS_KEY_ID': 'e6d28d7de535464da066c608726b4ee3',
     'AWS_SECRET_ACCESS_KEY': '2fc6029cc5d799b6c572c0e730d6bc912c047cb1dddce409',
     'MLFLOW_S3_ENDPOINT_URL': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
     'MLFLOW_TRACKING_URI': 'http://mlflow:5000'},
    'deployed_asset_type': 'model',
    'hardware_spec': {'id': 'b128f957-581d-46d0-95b6-8af5cd5be580',
     'name': 'XXS',
     'num_nodes': 1},
    'name': 'energyloss_111523_v2_three_kpis_loss_rate_joe',
    'online': {},
    'space_id': 'a0521ce5-5547-41e2-8495-66012d7e8780',
    'status': {'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'},
     'serving_url

In [50]:
deployment_endpoint = summary_data['summary']['deployment_details']['entity']['status']

In [51]:
deployment_endpoint

{'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'},
 'serving_urls': ['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'],
 'state': 'ready',
 'cp4d_urls': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-15',
 'cp4d_url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-16'}

In [53]:
deployment_endpoint['serving_urls']

['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions']

In [54]:
deployment_endpoint['cp4d_url'] = deployment_endpoint['serving_urls'][0]

In [55]:
deployment_endpoint['cp4d_url']

'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'

In [56]:
from datetime import datetime
# Get the current date
current_date = datetime.now()

# Format the date and create the string
version_string = f"?version={current_date.strftime('%Y-%m-%d')}"

In [57]:
deployment_endpoint['cp4d_url'] = deployment_endpoint['cp4d_url'] + version_string

In [58]:
deployment_endpoint

{'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'},
 'serving_urls': ['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'],
 'state': 'ready',
 'cp4d_urls': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-15',
 'cp4d_url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-16'}

In [59]:
with open("../config/deployment_info_three_kpis_loss_rate.yml","w") as file:
    yaml.dump(deployment_endpoint, file)